[为什么机器能够学习——PAC Learnability](http://www.cnblogs.com/wacc/p/4338437.html)

而在PAC学习的框架中，一个概念是强可学习的充要条件是这个概念是弱可学习的。


In [10]:
#参考公式原理：统计学习方法 例8.1
import numpy as np
import matplotlib.pyplot as plt

x = np.array([0,1,2,3,4,5,6,7,8,9])
y = np.array([1,1,1,-1,-1,-1,1,1,1,-1])

#弱分类器
def weakClassifier(x,threshold,reverse):
    #输入x，y
    #输出误差最小的阈值
    m=x.shape[0]
    y_t = np.ones(m)
    y_t[x>threshold] = -1
    if reverse:y_t=y_t*-1
    return y_t

def bestThreshold(x,y,D):
    #输入x，y
    #输出误差最小的阈值
    m=len(y)
    loss_min = float('inf')
    for i in range(min(x),max(x)+2):#选择阈值
        
        y_t = np.ones(m)
        y_t[x>=i] = -1
        loss = np.sum(D*np.abs(y-y_t))/m 
        if loss<loss_min:
            loss_min = loss
            best_v = i-0.5 #因为取的>= 所以取和小的值的中间值
            reverse = 0
            
        y_t_re = np.ones(m)*-1
        y_t_re[x>=i] = 1
        loss = np.sum(D*np.abs(y-y_t_re))/m 
        if loss<loss_min:
            loss_min = loss
            best_v = i-0.5 
            reverse = 1
            
    return best_v ,reverse

D = np.ones(len(y))*(1/len(y))  #数据权重
t,_ = bestThreshold(x,y,D)
print(t)
weakClassifier(x,t,reverse=0)

2.5


array([ 1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1.])

In [40]:
def AdaBoost(x,y,weakClassifier):
    #classNums为分类器的数量，即迭代次数
    #初始化
    num = len(y)
    w = 1/num   
    D = np.ones(num)*w  #数据权重
    alpha = [] #分类器权重
    em=1.0
    th_init,reverse = bestThreshold(x,y,D)
    G = [th_init] #存储分类器结果
    Rev = [reverse]
    while 1:
        y_pre_t = weakClassifier(x,G[-1],reverse)

        em = np.sum(D[y_pre_t!=y])
        alpha.append(0.5*np.log((1-em)/em))
        #UPDATE
        Z = np.sum(D*np.exp(-alpha[-1]*y*y_pre_t))
        D = D/Z * np.exp(-alpha[-1]*y*y_pre_t)
        v,reverse=bestThreshold(x,y,D)
        G.append(v)
        Rev.append(reverse)
        
        f = 0
        for i in range(len(alpha)):
            f += alpha[i]*weakClassifier(x,G[i],Rev[i])
        y_pre = np.sign(f)
        if ((y_pre-y)==0).all():
            break       
    return y_pre

AdaBoost(x,y,weakClassifier)

array([ 1.,  1.,  1., -1., -1., -1.,  1.,  1.,  1., -1.])